In [ ]:
# add signatures

ln <- load("../../exp_res_ks_20191002.RData")

ln

ln2 <- load("signature_list_twas_significant.RData")

head(up_signature_df)

signature_list_twas_significant

source("ks_connectivity.R")
source("correlation_connectivity.R")

# main function ks score
ks_score <- function(up_signature, down_signature, rank_matrix, compound_name) {
    n = 1
    connectivity_score_vec <- c()
    gene_rank_bing <- rank_matrix
    for (i in 1:ncol(gene_rank_bing)) {
        if (i %% 500 == 0)  { print(paste0("INFO: ", i, " Instances."))}
        pert_vec <- gene_rank_bing[, i]
        up_signature <- up_signature[up_signature %in% pert_vec]   
        down_signature <- down_signature[down_signature %in% pert_vec]
        # get teh rank
        up_v <- match(up_signature, pert_vec)
        down_v <- match(down_signature, pert_vec)
        tmp_s <- .s(up_v, down_v, nrow(gene_rank_bing))
        connectivity_score_vec <- c(connectivity_score_vec, tmp_s)
        
    }
    output_df <- data.frame(compound_name = compound_name, connectivity_score = connectivity_score_vec)
    return(output_df)
    
}




#ks_score_vec <- ks_score(up_signature_df$entrezgene_id, down_signature_df$entrezgene_id, gene_rank_bing, compound_name)

source("../utilities//gene_id.R")

id_map(grch38$symbol[4:9], from_ref = "symbol", to_ref = "entrez")

library(annotables)
library(dplyr)

# map the id
id_map <- function(id_vec, from_ref, to_ref, annotable = grch38) {
    id_df <- data.frame(raw_id = id_vec)
    colnames(id_df) <- from_ref
    id_df[, from_ref] <- as.character(id_df[, from_ref])
    by_str <- rlang::set_names(quo_name(from_ref), quo_name(from_ref))
    output_df <- id_df %>%
        left_join(annotable, by = by_str) %>%
        distinct_(from_ref, .keep_all = T) %>%
        select_(to_ref)
    return(output_df[, to_ref])                              
    
} 

extract_signature <- function(df) {
    up_idx <- df$zscore > 0
    down_idx <- !up_idx
    up_df <- df[up_idx, ]
    down_df <- df[down_idx, ]
    up_df$entrez <- id_map(up_df$gene, from_ref = "symbol", to_ref = "entrez")
    head(up_df)
    down_df$entrez <- id_map(down_df$gene, from_ref = "symbol", to_ref = "entrez")
    up_df <- up_df[complete.cases(up_df),]
    down_df <- down_df[complete.cases(down_df),]
    output_list <- list(up_df = up_df, down_df = down_df)
    return(output_list)
}

load("../../signature_list_twas_significant_unique.RData")
df <- signature_list_twas_significant_unique[["FC"]]


test_outputlist <- extract_signature(signature_list_twas_significant_unique[["FC"]])

test_outputlist$up_df

head(up_signature_df)

merged_up_signature_df <- data.frame(entrez = up_signature_df$entrezgene_id, pvalue_adj = up_signature_df$P_adjusted.x)
merged_down_signature_df <- data.frame(entrez = down_signature_df$entrezgene_id, pvalue_adj = down_signature_df$P_adjusted.x)




head(merged_up_signature_df)

merged_up_signature_df <- data.frame(entrez = up_signature_df$entrezgene_id, pvalue_adj = up_signature_df$P_adjusted.x)
merged_down_signature_df <- data.frame(entrez = down_signature_df$entrezgene_id, pvalue_adj = down_signature_df$P_adjusted.x)



merge_signature <- function(merged_up_signature_df, merged_down_signature_df, trait_str_vec) {
    for (trait_str in trait_str_vec) {
        cat(trait_str)
        output_list <- extract_signature(signature_list_twas_significant_unique[[trait_str]])
        tmp_up_df <- output_list$up_df
        tmp_down_df <- output_list$down_df
        tmp_up_df <- tmp_up_df[,c("entrez", "pvalue_adj")]
        tmp_down_df <- tmp_down_df[,c("entrez", "pvalue_adj")]
        merged_up_signature_df <- rbind(merged_up_signature_df, tmp_down_df)
        merged_down_signature_df <- rbind(merged_down_signature_df, tmp_up_df)
    }
    merged_up_signature_df <- merged_up_signature_df %>% arrange(pvalue_adj)
    merged_down_signature_df <- merged_down_signature_df %>% arrange(pvalue_adj)
    output_list <- list(merged_up_signature_df = merged_up_signature_df, merged_down_signature_df = merged_down_signature_df)
    return(output_list)
    
    
}



trait_vec <- c("Fatty_liver", "FC", "T2D", "CAD")
test_list <- merge_signature(merged_up_signature_df, merged_down_signature_df, trait_vec)

trait_vec <- c("Fatty_liver", "FC", "T2D", "CAD", "BMI_GIANT_2015")
test_list_2 <- merge_signature(merged_up_signature_df, merged_down_signature_df, trait_vec)

test_list_2$merged_up_signature_df

test_list_2$merged_down_signature_df

ks_score_vec <- ks_score(test_list$merged_up_signature_df$entrez, test_list$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

head(ks_score_vec)

ks_score_vec_2 <- ks_score(test_list_2$merged_up_signature_df$entrez, test_list_2$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

head(ks_score_vec_2)

check_overlap <- function(gsea_df, nash_drugs) {
    lincs_drugs <- tolower(gsea_df$compound_name)

    midx <- na.omit(match(nash_drugs, lincs_drugs))
    overlapped_drugs <- lincs_drugs[midx]
    sum_overlap <- ifelse(lincs_drugs %in% overlapped_drugs, 1, 0)
    #sum_overlap_top5 <- ifelse(lincs_drugs[(5774-289+1):5774] %in% overlapped_drugs, 1, 0)
    sum_overlap_top5 <- ifelse(lincs_drugs[1:289] %in% overlapped_drugs, 1, 0)
    #sum_overlap_top10 <- ifelse(lincs_drugs[(5774-578+1):5774] %in% overlapped_drugs, 1, 0)
    sum_overlap_top10 <- ifelse(lincs_drugs[1:578] %in% overlapped_drugs, 1, 0)
    sum_overlap_top_all <- ifelse(lincs_drugs %in% overlapped_drugs, 1, 0)
    return(c(sum(sum_overlap_top5), sum(sum_overlap_top10)))
}



cor_score_vec <- correlation_score_merged(test_list$merged_up_signature_df$entrez, test_list$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

cor_score_vec_2 <- correlation_score_merged(test_list_2$merged_up_signature_df$entrez, test_list_2$merged_down_signature_df$entrez, gene_rank_bing, compound_name)

drug_str <- "ACARBOSE AMLODIPINE ATORVASTATIN BETAINE CAFFEINE CENICRIVIROC CHOLECALCIFEROL DABIGATRAN DAPAGLIFLOZIN EMRICASAN ERGOCALCIFEROL EXENATIDE EZETIMIBE FENOFIBRATE GLIMEPIRIDE INSULIN GLARGINE INSULIN HUMAN PRADIGASTAT K-877 PX-102 LIRAGLUTIDE QUINIDINE LOSARTAN ROFLUMILAST METFORMIN ROSIGLITAZONE METRELEPTIN ROSUVASTATIN MGL-3196 SELADELPAR MIDAZOLAM SELONSERTIB SEMAGLUTIDE MT-3995 MUROMONAB-CD3 SIMTUZUMAB NAMODENOSON SITAGUPTIN NIVOCASAN SPIRONOLACTONE OBETICHOLIC ACID TELMISARTAN PENTOXIFYLUNE TOFOGLIFLOZIN PERINDOPRIL URSODIOL PIOGLITAZONE"

nash_drugs <- tolower(unlist(stringr::str_split(drug_str, " ")))

length(nash_drugs)



check_overlap(ks_score_vec, nash_drugs)

check_overlap(ks_score_vec_2, nash_drugs)

check_overlap(cor_score_vec, nash_drugs)

check_overlap(cor_score_vec_2, nash_drugs)

head(cor_score_vec)

head(cor_score_vec_2)

enrichment_phyper <- function(top_5_count, top_10_count, overlapped_count, total_count ) {
    p_5 <- phyper(top_5_count, overlapped_count, total_count-overlapped_count, round(total_count*0.05), lower.tail = F)
    p_10 <- phyper(top_5_count, overlapped_count, total_count-overlapped_count, round(total_count*0.10), lower.tail = F)
    return(c(p_5, p_10))
} 

overlapped_count <- 30
total_count <- 5774


enrichment_phyper(5, 6, overlapped_count, total_count)

trait_vec <- c("Fatty_liver", "FC")
test_list_close <- merge_signature(merged_up_signature_df, merged_down_signature_df, trait_vec)